In [ ]:
# Imports

import pandas as pd
import sqlite3
import shutil
import tempfile
from pathlib import Path

## Make a temporary copy of the Messages database

In [ ]:
src = Path("~/Library/Messages/chat.db").expanduser()
tmp_db = Path(tempfile.gettempdir()) / "chat_copy.db"
try:
    shutil.copy2(src, tmp_db)  # requires read permission on Messages.db
except PermissionError as e:
    raise RuntimeError(
        f"Cannot read {src}. Grant Full Disk Access to the Python/Jupyter process (System Settings → Privacy & Security → Full Disk Access) and re-run."
    ) from e

## Connect to the temporary database

In [ ]:
con = sqlite3.connect(tmp_db)

In [ ]:
cur = con.cursor()
res = cur.execute("SELECT sql FROM sqlite_master WHERE type='table'")
for r in res.fetchall():
    print(r)
res = cur.execute("SELECT * FROM message LIMIT 1")
print(res.fetchall())
res.close()
cur.close()

In [ ]:
pd.read_sql("SELECT text, subject, date, ck_chat_id FROM message WHERE ck_chat_id IS NOT NULL ORDER BY date DESC LIMIT 5", con)

In [ ]:
messages_df = pd.read_sql("SELECT * FROM message WHERE text IS NOT NULL AND text != '￼' AND ck_chat_id IS NOT NULL ORDER BY date DESC LIMIT 5", con)

# Convert the date column from Apple Absolute timestamp to a human-readable format
messages_df['date'] = pd.to_datetime(messages_df['date'] + 978307200000000000, unit='ns')

# print(messages_df.columns)
for col in messages_df.columns:
    print(f"{col}: {messages_df[col].iloc[0]}")


In [ ]:
chats_df = pd.read_sql("SELECT * FROM chat ORDER BY last_read_message_timestamp DESC LIMIT 1", con)

# # Convert the date column from nanoseconds since epoch to a human-readable format
# chats_df['date'] = pd.to_datetime(chats_df['date'], unit='ns')

# print(chats_df.columns)
for col in chats_df.columns:
    print(f"{col}: {chats_df[col].iloc[0]}")